In [1]:
import pandas as pd
from sklearn.metrics import RocCurveDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn import metrics
from collections import defaultdict
import joblib

import sys

import time
start_time = time.time()


infile='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/realtrainingtable/BL14_genepromSM_0.7.txt_REALTEST_bestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl'


gtcol='label2'

##### carefully input the model folder#####

modelfol='/Users/irffanalahi/Research/Research_update/SoftRD/largerEXP/genepromdelta.7/Randomforest/developing/realtrainingtable/BL14_genepromSM_0.7.txt_REALTRAINbestref.txt_result_dupindex_binnedstats.pkl_totalformatteddata.pkl_model_10000'


indf=pd.read_pickle(infile)

indf.head()

,index,CD4-others,CD8-others,nB-others,NK-others,Mn-others,mNeu-others,m8-others,DC-others,Eo-others,...,maxscore_beforeCpGweight,maxscoredCT,maxscore,deltabasedfragassignment,filename,label,Last,label2,deltaMedian,deltaMean
0,CD4.HWI-7001432L_73:1:1301:17083:62471#12,0.740282,0.736333,-0.280282,-0.116949,-0.260179,-0.260538,0.168795,-0.280282,-0.268436,...,0.740282,CD4-others,0.740282,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,CD4,HWI-7001432L_73:1:1301:17083:62471#12,CD4,-0.207500,3.965082e-18
1,CD4.HWI-7001432L_73:1:2103:15384:27344#12,1.470244,1.029782,-0.478269,-0.078731,-0.474679,-0.252833,-0.105295,-0.336474,-0.349756,...,1.470244,CD4-others,1.470244,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,CD4,HWI-7001432L_73:1:2103:15384:27344#12,CD4,-0.236141,-7.137148e-17
2,CD4.HWI-7001432L_74:1:1215:11529:42195#12,0.794564,0.657436,-0.185795,-0.150615,-0.164256,-0.154564,-0.022462,-0.185436,-0.093000,...,0.794564,CD4-others,0.794564,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,CD4,HWI-7001432L_74:1:1215:11529:42195#12,CD4,-0.151692,-4.956353e-17
3,CD4.HWI-ST1450_140:1:1307:21239:47394#12,0.757333,0.385795,-0.280103,-0.205436,-0.235231,-0.211897,0.096821,-0.245282,-0.280103,...,0.757333,CD4-others,0.378667,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,CD4,HWI-ST1450_140:1:1307:21239:47394#12,CD4,-0.164154,1.784287e-17
4,CD4.HWI-ST1450_140:1:2215:5251:69917#12,0.738205,-0.133744,-0.264769,-0.271949,-0.237487,-0.201590,0.289128,-0.217744,-0.234795,...,0.738205,CD4-others,0.738205,CD4-others,CD4_fivemixgeneproNotpicard_NR_767146_insilmix...,CD4,HWI-ST1450_140:1:2215:5251:69917#12,CD4,-0.204103,-3.965082e-18


In [2]:
features=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others','LENhypoCpG', 'LENhyperCpG', 'total_cpg', 'LENhypoCpG_BY_total_cpg','deltaMedian', 'deltaMean']

In [3]:
scorecolumns=['CD4-others', 'CD8-others', 'nB-others', 'NK-others','Mn-others', 'mNeu-others', 'm8-others', 'DC-others', 'Eo-others','Tr-others', 'm4-others', 'Mg-others', 'Er-others', 'mB-others']

In [4]:
celltypeAUC=defaultdict(list)

for score in scorecolumns:
    
    tempct=score.replace('-others','')
    
 
  
    indfsingleCT=indf[(indf[gtcol]==tempct) | (indf[gtcol]=='NOT'+tempct)].copy()
    
    
    
    indfsingleCT.loc[indfsingleCT[gtcol]==tempct,gtcol]=1
    indfsingleCT.loc[indfsingleCT[gtcol]=='NOT'+tempct,gtcol]=0
    

    
    indfsingleCT[gtcol]=indfsingleCT[gtcol].astype(int)
    
    
  
    testY=indfsingleCT[gtcol]
    testX=indfsingleCT[features]
    
    RDF_classifier =joblib.load(modelfol+"/"+tempct+".pkl")
    
    predict_test=RDF_classifier.predict_proba(testX)
    
    
    
    
    fpr, tpr, thresholds=metrics.roc_curve(testY,predict_test[:,1],pos_label=1)


    tempAUC=metrics.auc(fpr, tpr)
    
    
   
    
   


    
    
    celltypeAUC[tempct]=celltypeAUC[tempct]+[tempAUC]

In [5]:
celltypeAUCdf=pd.DataFrame.from_dict(celltypeAUC, orient='index',columns=['AUC'])
celltypeAUCdf.index.name = 'Celltype'
celltypeAUCdf.to_csv(infile+"_RFAUC.txt",sep="\t")

In [6]:
end_time = time.time()

time_elapsed = (end_time - start_time)

print(time_elapsed)

50.36695408821106
